In [1]:
import networkx as nx
import numpy as np
from operator import sub, add
from collections import Counter

In [2]:
with open("inputs/day12", "r") as f:
  puzzle_input =  np.array([[*l] for l in f.read().strip().split("\n")])

# Build graph
graph = nx.create_empty_copy(nx.grid_graph(puzzle_input.shape))
for iy, ix in np.ndindex(puzzle_input.shape):
  try:
    if puzzle_input[iy][ix] == puzzle_input[iy][ix+1]:
      graph.add_edge((iy,ix),(iy,ix+1))
  except:
    pass

  try:
    if puzzle_input[iy][ix] == puzzle_input[iy+1][ix]:
      graph.add_edge((iy,ix),(iy+1,ix))
  except:
    pass

In [3]:
# Problem 1
res = 0
for cc in nx.connected_components(graph):
  size = len(cc)
  circum = 0
  for n in cc:
    circum += (4-len(list(graph.neighbors(n))))
  res += size * circum

print(res)

1546338


In [4]:
# Problem 2
def check_diagonal(n, n1, n2, cc):
  missing_corner = tuple(map(add, tuple(map(sub, n, n1)), tuple(map(sub, n, n2))))
  return tuple(map(sub, n, missing_corner)) in cc

res = 0
for cc in nx.connected_components(graph):
  size = len(cc)
  num_corners = 0
  for n in cc:
    add_c = 0
    match len(list(graph.neighbors(n))):
      case 0:
        add_c = 4
      case 1:
        add_c = 2
      case 2:
        neighbors = list(graph.neighbors(n))
        if np.subtract(neighbors[0], neighbors[1]).all():
          if check_diagonal(n, neighbors[0], neighbors[1], cc):
            add_c = 1
          else:
            add_c = 2
        else:
          add_c = 0
      case 3:
        neighbors = list(graph.neighbors(n))
        y_coords = Counter(list(zip(*neighbors))[0])
        x_coords = Counter(list(zip(*neighbors))[1])

        diag_neighbors = set(neighbors)
        for neighbor in neighbors:
          if y_coords[neighbor[0]] <= 1 and x_coords[neighbor[1]] <= 1:
            diag_neighbors.remove(neighbor)
            break
        diag_neighbors = list(diag_neighbors)

        add_c += not check_diagonal(n, neighbor, diag_neighbors[0], cc)
        add_c += not check_diagonal(n, neighbor, diag_neighbors[1], cc)
      case 4:
        add_c += not check_diagonal(n, tuple(map(add, n, (-1,0))), tuple(map(add, n, (0,-1))), cc)
        add_c += not check_diagonal(n, tuple(map(add, n, (0,-1))), tuple(map(add, n, (1,0))), cc)
        add_c += not check_diagonal(n, tuple(map(add, n, (1,0))), tuple(map(add, n, (0,1))), cc)
        add_c += not check_diagonal(n, tuple(map(add, n, (0,1))), tuple(map(add, n, (-1,0))), cc)

    num_corners += add_c

  res += size * num_corners

print(res)

978590
